# Goal
## Original Questions
- Can we use Chronic Health Conditions to accurately predict Health Care Access?
- Are there Demographic clusters that are disproportionately affected by Chronic Health Conditions?
- Can unsupervised learning methods reveal distinct clusters that account for the bulk of Chronic Health Conditions?

### Questions:
- I have gotten a bit hung up bc as worded 2 and 3 seem to be asking same thing?
- were there established chronic health conditions?
- established demographic features?
- what are the features used in RQ1, and RQ3?

#### Rough Plan:
Question: Refinement:
Are there diagnostically useful demographic clusters that indicate chronic health conditions?
- Are there demographic clusters that strongly indicate certain chronic health conditions?
- Can we predict chronic health conditions from demographics, and how does a ML model compare with simpler cluster membership?

##### Part 1: Clustering
- Cluster the demographic features of the BRFSS data
- Visualize clusters and prevalence of chronic health conditions within each cluster
    - what chronic health conditions to use?
    - VISUAL: Clustering results
    - VISUAL:  Heatmaps of cluster membership vs chronic health conditions
- Run statistical tests to determine if certain clusters are significantly more affected by chronic health conditions
    - Translation - test the strength of correlation between cluster membership and chronic health conditions
    - Does being a member of a cluster correlate with having a chronic health condition?
#### Part 2: Prediction of Chronic Health Conditions From Demographics via DL Model. Inversion of Question 3
- use cluster labels as features?
- compare performance of a deep learning model vs simpler clustering membership



### Misc
- potential 'linchpin' variables given we are clustering on demographics (and ran random forest on demographics)


In [ ]:
import os
import sys
from IPython import get_ipython
import logging

import subprocess
import sys

try:
    import kmodes
    print("kmodes already installed")
except ImportError:
    print("Installing kmodes...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "kmodes"])
    import kmodes
    print("kmodes installed successfully")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def is_colab():
    return 'google.colab' in str(get_ipython())

# Set up environment and paths
if is_colab():
    print("Running in Google Colab")

    # Clone the repository if not already cloned
    if not os.path.exists('dat490'):
        import subprocess
        print("Cloning repository...")
        subprocess.run(['git', 'clone', 'https://github.com/sksizer/dat490.git'], check=True)
        print("Repository cloned successfully")

    # Add the repository to Python path for imports
    sys.path.insert(0, '/content/dat490')

    # Set paths to use data from the cloned repository
    BFRSS_DATA_PATH = 'dat490/data/LLCP2023.parquet'
    BFRSS_CODEBOOK_PATH = 'dat490/data/codebook_USCODE23_LLCP_021924.HTML'
    BFRSS_DESC_PATH = 'dat490/data/LLCP2023_desc.parquet'  # Additional metadata file if needed
else:
    print("Running in local environment")

    # Add parent directory to path for dat490 module imports
    sys.path.insert(0, os.path.abspath('..'))

    # Use local data paths
    BFRSS_DATA_PATH = '../data/LLCP2023.parquet'
    BFRSS_CODEBOOK_PATH = '../data/codebook_USCODE23_LLCP_021924.HTML'
    BFRSS_DESC_PATH = '../data/LLCP2023_desc.parquet'  # Additional metadata file if needed

# Verify files exist
print(f"\\nData path: {BFRSS_DATA_PATH}")
print(f"Codebook path: {BFRSS_CODEBOOK_PATH}")

if not os.path.exists(BFRSS_DATA_PATH):
    raise FileNotFoundError(f"Data file not found at {BFRSS_DATA_PATH}")

if not os.path.exists(BFRSS_CODEBOOK_PATH):
    raise FileNotFoundError(f"Codebook file not found at {BFRSS_CODEBOOK_PATH}")

print("\\nAll required files found!")
logger.info('Environment setup complete')

##################################
# Load BFRSS data and metadata using the new wrapper
from dat490 import load_bfrss

# Single function call to load everything
# exclude_desc_columns=True will exclude _DESC columns from metadata generation
bfrss = load_bfrss(exclude_desc_columns=True)

# Get a copy of the raw DataFrame
bfrss_raw_df = bfrss.cloneDF()
bfrss_raw_df.info()

DEMOGRAPHIC_FEATURE_COLUMNS = [
    # Demographics section columns (13 total)
    # Demographics section columns (13 total)
    'MARITAL',    # https://singular-eclair-6a5a16.netlify.app/columns/MARITAL
    'EDUCA',      # https://singular-eclair-6a5a16.netlify.app/columns/EDUCA
    'RENTHOM1',   # https://singular-eclair-6a5a16.netlify.app/columns/RENTHOM1
    # 'NUMHHOL4',   # https://singular-eclair-6a5a16.netlify.app/columns/NUMHHOL4
    # 'NUMPHON4',   # https://singular-eclair-6a5a16.netlify.app/columns/NUMPHON4
    # 'CPDEMO1C',   # https://singular-eclair-6a5a16.netlify.app/columns/CPDEMO1C
    'VETERAN3',   # https://singular-eclair-6a5a16.netlify.app/columns/VETERAN3
    'EMPLOY1',    # https://singular-eclair-6a5a16.netlify.app/columns/EMPLOY1
    # 'CHILDREN',   # https://singular-eclair-6a5a16.netlify.app/columns/CHILDREN
    'INCOME3',    # https://singular-eclair-6a5a16.netlify.app/columns/INCOME3
    # 'PREGNANT',   # https://singular-eclair-6a5a16.netlify.app/columns/PREGNANT
    'SEXVAR',    # https://singular-eclair-6a5a16.netlify.app/columns/SEXVAR
    '_HISPANC', # https://singular-eclair-6a5a16.netlify.app/columns/_HISPANC # Calculated but not sure from what
    # '_CRACE1',    # https://singular-eclair-6a5a16.netlify.app/columns/_CRACE1 # Child race
    '_IMPRACE',   # https://singular-eclair-6a5a16.netlify.app/columns/_IMPRACE
    # '_AGE80',     # https://singular-eclair-6a5a16.netlify.app/columns/_AGE80
]




# K-Modes Analysis of BRFSS Data

K-Modes clustering is an extension of K-Means designed for categorical data. Instead of using means to define cluster centers, K-Modes uses modes (most frequent values) and measures dissimilarity using the number of mismatches between data points.

In [ ]:
import pandas as pd
from kmodes.kmodes import KModes

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from kmodes.kmodes import KModes
from sklearn.preprocessing import LabelEncoder

def kmode_analysis(df: pd.DataFrame, feature_cols: list, max_clusters: int = 6):
    data = df[feature_cols].copy()

    # Encode categorical variables
    encoders = {}
    for col in data.columns:
        if data[col].dtype == 'object' or data[col].dtype.name == 'category':
            le = LabelEncoder()
            data[col] = data[col].fillna(data[col].mode()[0])
            data[col] = le.fit_transform(data[col])
            encoders[col] = le
        else:
            data[col] = data[col].fillna(data[col].mean())

    # Elbow Method
    cost = []
    for k in range(1, max_clusters + 1):
        km = KModes(n_clusters=k, init='Huang', n_init=5, verbose=0)
        km.fit_predict(data)
        cost.append(km.cost_)

    plt.figure(figsize=(8, 4))
    plt.plot(range(1, max_clusters + 1), cost, marker='o')
    plt.title('Elbow Method - Cost vs. Number of Clusters')
    plt.xlabel('Number of clusters')
    plt.ylabel('Cost')
    plt.grid(True)
    plt.show()

    # Fit the best model (you can tune n_clusters here if needed)
    optimal_k = 2  # or set based on elbow result
    km = KModes(n_clusters=optimal_k, init='Huang', n_init=5, verbose=0)
    clusters = km.fit_predict(data)
    df['Cluster'] = clusters

    # Cluster sizes
    cluster_counts = df['Cluster'].value_counts().sort_index()
    plt.figure(figsize=(6, 4))
    sns.barplot(x=cluster_counts.index, y=cluster_counts.values)
    plt.title('Cluster Size Distribution')
    plt.xlabel('Cluster')
    plt.ylabel('Number of Records')
    plt.show()

    # Cluster composition heatmap
    cluster_profiles = pd.DataFrame(data)
    cluster_profiles['Cluster'] = clusters
    summary = cluster_profiles.groupby('Cluster')[feature_cols].agg(lambda x: x.value_counts().index[0])

    plt.figure(figsize=(10, 6))
    sns.heatmap(summary.applymap(float), annot=True, fmt=".0f", cmap="YlGnBu")
    plt.title("Cluster Centroids (Encoded)")
    plt.xlabel("Features")
    plt.ylabel("Cluster")
    plt.show()

    # Print centroids and return DataFrame
    print("Cluster centroids:")
    print(km.cluster_centroids_)
    return df

kmode_analysis(bfrss_raw_df, DEMOGRAPHIC_FEATURE_COLUMNS)

# Machine Learn Model to Predict Chronic Health Conditions from Demographics



In [ ]:
# Data preprocessing function
def preprocess_data_for_ml(df, feature_cols, target_col, encode_target=True):
    """
    Preprocess data for machine learning:
    - Handle missing values
    - Encode categorical variables
    - Scale numerical features
    
    Returns:
    - X: Preprocessed features
    - y: Target variable
    - encoders: Dictionary of label encoders for categorical features
    - scaler: StandardScaler object
    """
    # Create a copy of the dataframe
    data = df.copy()
    
    # Filter to only include rows with valid target values
    if encode_target:
        # For categorical targets, filter out missing/refused/don't know responses
        valid_mask = data[target_col].notna()
        if data[target_col].dtype == 'object' or data[target_col].dtype.name == 'category':
            # Common invalid responses in BRFSS data
            invalid_responses = ['7', '9', 'Dont know', 'Refused', 'Missing']
            valid_mask = valid_mask & (~data[target_col].astype(str).isin(invalid_responses))
    else:
        valid_mask = data[target_col].notna()
    
    data = data[valid_mask]
    
    # Prepare features
    X = data[feature_cols].copy()
    y = data[target_col].copy()
    
    # Dictionary to store encoders
    encoders = {}
    
    # Encode categorical features
    for col in X.columns:
        if X[col].dtype == 'object' or X[col].dtype.name == 'category':
            le = LabelEncoder()
            # Fill missing values with mode
            mode_val = X[col].mode()[0] if len(X[col].mode()) > 0 else 'missing'
            X[col] = X[col].fillna(mode_val)
            X[col] = le.fit_transform(X[col].astype(str))
            encoders[col] = le
        else:
            # Fill missing numerical values with mean
            X[col] = X[col].fillna(X[col].mean())
    
    # Encode target variable if needed
    if encode_target and (y.dtype == 'object' or y.dtype.name == 'category'):
        target_encoder = LabelEncoder()
        y = target_encoder.fit_transform(y.astype(str))
        encoders['target'] = target_encoder
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, encoders, scaler

# Function to create a binary target from chronic condition responses
def create_binary_target(series, positive_values=['1', 'Yes']):
    """
    Convert BRFSS response to binary (0/1) target.
    Typically, '1' or 'Yes' means the person has the condition.
    """
    return series.astype(str).isin([str(v) for v in positive_values]).astype(int)

In [ ]:
# Install TensorFlow if needed
try:
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    print("Installing TensorFlow...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow"])
    import tensorflow as tf
    print(f"TensorFlow installed successfully. Version: {tf.__version__}")

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import numpy as np

In [ ]:
# Subsampling and Stability Testing for K-Modes

import numpy as np
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

def subsample_kmode_stability(df: pd.DataFrame, 
                            feature_cols: list, 
                            n_clusters: int = 2,
                            n_iterations: int = 10,
                            subsample_ratio: float = 0.8,
                            random_state: int = 42):
    """
    Perform stability testing of k-modes clustering through subsampling.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_cols : list
        List of feature columns to use for clustering
    n_clusters : int
        Number of clusters for k-modes
    n_iterations : int
        Number of subsampling iterations
    subsample_ratio : float
        Proportion of data to sample in each iteration
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    dict : Dictionary containing stability metrics and results
    """
    
    np.random.seed(random_state)
    
    # Prepare data
    data = df[feature_cols].copy()
    
    # Encode categorical variables
    encoders = {}
    for col in data.columns:
        if data[col].dtype == 'object' or data[col].dtype.name == 'category':
            le = LabelEncoder()
            data[col] = data[col].fillna(data[col].mode()[0] if len(data[col].mode()) > 0 else 'missing')
            data[col] = le.fit_transform(data[col])
            encoders[col] = le
        else:
            data[col] = data[col].fillna(data[col].mean())
    
    # Storage for results
    results = {
        'cluster_assignments': [],
        'centroids': [],
        'costs': [],
        'ari_scores': [],
        'nmi_scores': [],
        'stability_scores': []
    }
    
    # Reference clustering on full dataset
    km_ref = KModes(n_clusters=n_clusters, init='Huang', n_init=5, verbose=0)
    ref_clusters = km_ref.fit_predict(data)
    ref_centroids = km_ref.cluster_centroids_
    
    print(f"Running {n_iterations} iterations with {subsample_ratio*100:.0f}% subsampling...")
    
    for i in range(n_iterations):
        # Subsample data
        n_samples = int(len(data) * subsample_ratio)
        sample_idx = np.random.choice(len(data), n_samples, replace=False)
        data_sample = data.iloc[sample_idx]
        
        # Run k-modes on subsample
        km = KModes(n_clusters=n_clusters, init='Huang', n_init=5, verbose=0)
        clusters = km.fit_predict(data_sample)
        
        # Store results
        results['cluster_assignments'].append(clusters)
        results['centroids'].append(km.cluster_centroids_)
        results['costs'].append(km.cost_)
        
        # Calculate stability metrics against reference (for overlapping samples)
        ref_clusters_sample = ref_clusters[sample_idx]
        ari = adjusted_rand_score(ref_clusters_sample, clusters)
        nmi = normalized_mutual_info_score(ref_clusters_sample, clusters)
        
        results['ari_scores'].append(ari)
        results['nmi_scores'].append(nmi)
        
        if (i + 1) % 5 == 0:
            print(f"  Completed iteration {i+1}/{n_iterations}")
    
    # Calculate pairwise stability between iterations
    print("\\nCalculating pairwise stability...")
    for i in range(n_iterations):
        for j in range(i+1, n_iterations):
            # Find common indices between two subsamples
            # Since we're using random sampling, we need to track indices
            # For simplicity, we'll calculate stability on the full dataset predictions
            pass
    
    # Summary statistics
    results['summary'] = {
        'mean_cost': np.mean(results['costs']),
        'std_cost': np.std(results['costs']),
        'mean_ari': np.mean(results['ari_scores']),
        'std_ari': np.std(results['ari_scores']),
        'mean_nmi': np.mean(results['nmi_scores']),
        'std_nmi': np.std(results['nmi_scores']),
        'reference_centroids': ref_centroids,
        'reference_clusters': ref_clusters
    }
    
    return results

# Run stability analysis
stability_results = subsample_kmode_stability(
    bfrss_raw_df, 
    DEMOGRAPHIC_FEATURE_COLUMNS,
    n_clusters=2,
    n_iterations=20,
    subsample_ratio=0.8
)

In [ ]:
# Advanced Stability Metrics with Pairwise Comparisons

def calculate_pairwise_stability(df: pd.DataFrame,
                               feature_cols: list,
                               n_clusters: int = 2,
                               n_iterations: int = 10,
                               subsample_ratio: float = 0.8,
                               random_state: int = 42):
    """
    Calculate pairwise stability between multiple k-modes runs with bootstrap confidence intervals.
    """
    
    np.random.seed(random_state)
    
    # Prepare data
    data = df[feature_cols].copy()
    
    # Encode categorical variables
    for col in data.columns:
        if data[col].dtype == 'object' or data[col].dtype.name == 'category':
            le = LabelEncoder()
            data[col] = data[col].fillna(data[col].mode()[0] if len(data[col].mode()) > 0 else 'missing')
            data[col] = le.fit_transform(data[col])
        else:
            data[col] = data[col].fillna(data[col].mean())
    
    # Store sample indices and cluster assignments
    sample_indices = []
    cluster_assignments = []
    centroids = []
    
    print(f"Running {n_iterations} k-modes iterations...")
    
    for i in range(n_iterations):
        # Subsample data
        n_samples = int(len(data) * subsample_ratio)
        sample_idx = np.random.choice(len(data), n_samples, replace=False)
        sample_indices.append(sample_idx)
        
        data_sample = data.iloc[sample_idx]
        
        # Run k-modes
        km = KModes(n_clusters=n_clusters, init='Huang', n_init=5, verbose=0)
        clusters = km.fit_predict(data_sample)
        
        cluster_assignments.append(clusters)
        centroids.append(km.cluster_centroids_)
        
        if (i + 1) % 5 == 0:
            print(f"  Completed iteration {i+1}/{n_iterations}")
    
    # Calculate pairwise stability
    print("\\nCalculating pairwise stability metrics...")
    pairwise_ari = np.zeros((n_iterations, n_iterations))
    pairwise_nmi = np.zeros((n_iterations, n_iterations))
    
    for i in range(n_iterations):
        for j in range(i, n_iterations):
            if i == j:
                pairwise_ari[i, j] = 1.0
                pairwise_nmi[i, j] = 1.0
            else:
                # Find common indices
                common_idx = np.intersect1d(sample_indices[i], sample_indices[j])
                
                if len(common_idx) > 0:
                    # Get positions in respective samples
                    pos_i = np.searchsorted(sample_indices[i], common_idx)
                    pos_j = np.searchsorted(sample_indices[j], common_idx)
                    
                    # Compare cluster assignments for common samples
                    clusters_i = cluster_assignments[i][pos_i]
                    clusters_j = cluster_assignments[j][pos_j]
                    
                    ari = adjusted_rand_score(clusters_i, clusters_j)
                    nmi = normalized_mutual_info_score(clusters_i, clusters_j)
                    
                    pairwise_ari[i, j] = pairwise_ari[j, i] = ari
                    pairwise_nmi[i, j] = pairwise_nmi[j, i] = nmi
    
    # Calculate centroid stability
    centroid_distances = np.zeros((n_iterations, n_iterations))
    for i in range(n_iterations):
        for j in range(i, n_iterations):
            if i == j:
                centroid_distances[i, j] = 0.0
            else:
                # Calculate Hamming distance between centroids
                dist = 0
                for k in range(n_clusters):
                    dist += np.sum(centroids[i][k] != centroids[j][k])
                centroid_distances[i, j] = centroid_distances[j, i] = dist / (n_clusters * len(feature_cols))
    
    return {
        'pairwise_ari': pairwise_ari,
        'pairwise_nmi': pairwise_nmi,
        'centroid_distances': centroid_distances,
        'mean_ari': np.mean(pairwise_ari[np.triu_indices(n_iterations, k=1)]),
        'std_ari': np.std(pairwise_ari[np.triu_indices(n_iterations, k=1)]),
        'mean_nmi': np.mean(pairwise_nmi[np.triu_indices(n_iterations, k=1)]),
        'std_nmi': np.std(pairwise_nmi[np.triu_indices(n_iterations, k=1)]),
        'mean_centroid_dist': np.mean(centroid_distances[np.triu_indices(n_iterations, k=1)]),
        'std_centroid_dist': np.std(centroid_distances[np.triu_indices(n_iterations, k=1)])
    }

# Run pairwise stability analysis
pairwise_results = calculate_pairwise_stability(
    bfrss_raw_df,
    DEMOGRAPHIC_FEATURE_COLUMNS,
    n_clusters=2,
    n_iterations=15,
    subsample_ratio=0.8
)

In [ ]:
# Visualization Functions for Stability Results

def visualize_stability_results(results, title_prefix=""):
    """
    Create comprehensive visualizations for k-modes stability analysis.
    """
    
    # Set up the figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'{title_prefix}K-Modes Clustering Stability Analysis', fontsize=16)
    
    # 1. Stability Metrics Over Iterations (if from subsample_kmode_stability)
    if 'ari_scores' in results:
        ax = axes[0, 0]
        iterations = range(1, len(results['ari_scores']) + 1)
        
        ax.plot(iterations, results['ari_scores'], 'o-', label='ARI', markersize=8)
        ax.plot(iterations, results['nmi_scores'], 's-', label='NMI', markersize=8)
        
        # Add mean lines
        ax.axhline(y=results['summary']['mean_ari'], color='blue', linestyle='--', alpha=0.5)
        ax.axhline(y=results['summary']['mean_nmi'], color='orange', linestyle='--', alpha=0.5)
        
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Score')
        ax.set_title('Stability Metrics vs Reference Clustering')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_ylim(0, 1.05)
    
    # 2. Pairwise Stability Heatmap (if from calculate_pairwise_stability)
    if 'pairwise_ari' in results:
        ax = axes[0, 1]
        sns.heatmap(results['pairwise_ari'], annot=True, fmt='.2f', cmap='YlOrRd', 
                   vmin=0, vmax=1, square=True, ax=ax, cbar_kws={'label': 'ARI'})
        ax.set_title('Pairwise ARI Between Iterations')
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Iteration')
    
    # 3. Distribution of Stability Scores
    if 'pairwise_ari' in results:
        ax = axes[1, 0]
        
        # Extract upper triangle values (excluding diagonal)
        n_iter = results['pairwise_ari'].shape[0]
        ari_values = results['pairwise_ari'][np.triu_indices(n_iter, k=1)]
        nmi_values = results['pairwise_nmi'][np.triu_indices(n_iter, k=1)]
        
        # Create violin plots
        data_to_plot = [ari_values, nmi_values]
        positions = [1, 2]
        
        parts = ax.violinplot(data_to_plot, positions=positions, showmeans=True, showmedians=True)
        
        # Customize colors
        colors = ['#3498db', '#e74c3c']
        for pc, color in zip(parts['bodies'], colors):
            pc.set_facecolor(color)
            pc.set_alpha(0.7)
        
        ax.set_xticks(positions)
        ax.set_xticklabels(['ARI', 'NMI'])
        ax.set_ylabel('Score')
        ax.set_title('Distribution of Pairwise Stability Scores')
        ax.set_ylim(0, 1.05)
        ax.grid(True, axis='y', alpha=0.3)
        
        # Add summary statistics
        ax.text(1, 0.05, f'μ={np.mean(ari_values):.3f}\\nσ={np.std(ari_values):.3f}', 
                ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        ax.text(2, 0.05, f'μ={np.mean(nmi_values):.3f}\\nσ={np.std(nmi_values):.3f}', 
                ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 4. Centroid Stability
    if 'centroid_distances' in results:
        ax = axes[1, 1]
        
        # Create a histogram of centroid distances
        distances = results['centroid_distances'][np.triu_indices(results['centroid_distances'].shape[0], k=1)]
        
        ax.hist(distances, bins=20, edgecolor='black', alpha=0.7)
        ax.set_xlabel('Normalized Hamming Distance')
        ax.set_ylabel('Frequency')
        ax.set_title('Distribution of Centroid Distances')
        ax.grid(True, axis='y', alpha=0.3)
        
        # Add mean line
        mean_dist = np.mean(distances)
        ax.axvline(x=mean_dist, color='red', linestyle='--', linewidth=2, 
                  label=f'Mean = {mean_dist:.3f}')
        ax.legend()
    
    # Adjust layout
    plt.tight_layout()
    
    # Print summary statistics
    print("\\n=== Stability Analysis Summary ===")
    if 'summary' in results:
        print(f"Mean ARI: {results['summary']['mean_ari']:.3f} ± {results['summary']['std_ari']:.3f}")
        print(f"Mean NMI: {results['summary']['mean_nmi']:.3f} ± {results['summary']['std_nmi']:.3f}")
    elif 'mean_ari' in results:
        print(f"Mean Pairwise ARI: {results['mean_ari']:.3f} ± {results['std_ari']:.3f}")
        print(f"Mean Pairwise NMI: {results['mean_nmi']:.3f} ± {results['std_nmi']:.3f}")
        print(f"Mean Centroid Distance: {results['mean_centroid_dist']:.3f} ± {results['std_centroid_dist']:.3f}")
    
    return fig

# Visualize the stability results
if 'stability_results' in locals():
    visualize_stability_results(stability_results, "Subsample ")
    
if 'pairwise_results' in locals():
    visualize_stability_results(pairwise_results, "Pairwise ")

In [ ]:
# Quick Proof of Concept with Small Subsampling

def quick_stability_test(df: pd.DataFrame,
                        feature_cols: list,
                        n_clusters: int = 2,
                        n_iterations: int = 5,
                        subsample_size: int = 1000,  # Absolute sample size instead of ratio
                        random_state: int = 42):
    """
    Quick stability test with small absolute sample sizes for proof of concept.
    
    Parameters:
    -----------
    subsample_size : int
        Absolute number of samples to use (default 1000 for quick testing)
    """
    
    # Calculate the ratio based on absolute size
    total_rows = len(df)
    subsample_ratio = min(subsample_size / total_rows, 1.0)
    
    print(f"\\n=== Quick Stability Test ===")
    print(f"Total rows: {total_rows:,}")
    print(f"Sample size: {subsample_size:,} ({subsample_ratio*100:.1f}% of data)")
    print(f"Iterations: {n_iterations}")
    print(f"Clusters: {n_clusters}\\n")
    
    # Run the pairwise stability analysis with small samples
    results = calculate_pairwise_stability(
        df,
        feature_cols,
        n_clusters=n_clusters,
        n_iterations=n_iterations,
        subsample_ratio=subsample_ratio,
        random_state=random_state
    )
    
    return results

# Run quick test with very small sample
print("Running quick proof of concept with 1000 samples...")
quick_results = quick_stability_test(
    bfrss_raw_df,
    DEMOGRAPHIC_FEATURE_COLUMNS,
    n_clusters=2,
    n_iterations=5,
    subsample_size=1000  # Only 1000 samples for quick testing
)

# Visualize quick results
visualize_stability_results(quick_results, "Quick Test - ")

In [ ]:
# Comprehensive Stability Analysis with Multiple Sample Sizes

def multi_scale_stability_analysis(df: pd.DataFrame,
                                 feature_cols: list,
                                 n_clusters: int = 2,
                                 sample_sizes: list = [500, 1000, 5000, 10000],
                                 n_iterations: int = 10,
                                 random_state: int = 42):
    """
    Run stability analysis across multiple sample sizes to understand scaling behavior.
    """
    
    results_by_size = {}
    
    for sample_size in sample_sizes:
        print(f"\\n{'='*50}")
        print(f"Testing with sample size: {sample_size:,}")
        
        results = quick_stability_test(
            df,
            feature_cols,
            n_clusters=n_clusters,
            n_iterations=n_iterations,
            subsample_size=sample_size,
            random_state=random_state
        )
        
        results_by_size[sample_size] = results
    
    # Create summary plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Stability metrics vs sample size
    sizes = list(results_by_size.keys())
    mean_aris = [results_by_size[s]['mean_ari'] for s in sizes]
    std_aris = [results_by_size[s]['std_ari'] for s in sizes]
    mean_nmis = [results_by_size[s]['mean_nmi'] for s in sizes]
    std_nmis = [results_by_size[s]['std_nmi'] for s in sizes]
    
    ax1.errorbar(sizes, mean_aris, yerr=std_aris, marker='o', label='ARI', capsize=5, markersize=8)
    ax1.errorbar(sizes, mean_nmis, yerr=std_nmis, marker='s', label='NMI', capsize=5, markersize=8)
    
    ax1.set_xlabel('Sample Size')
    ax1.set_ylabel('Mean Stability Score')
    ax1.set_title('Stability vs Sample Size')
    ax1.set_xscale('log')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1.05)
    
    # Plot 2: Centroid distance vs sample size
    mean_dists = [results_by_size[s]['mean_centroid_dist'] for s in sizes]
    std_dists = [results_by_size[s]['std_centroid_dist'] for s in sizes]
    
    ax2.errorbar(sizes, mean_dists, yerr=std_dists, marker='d', color='green', capsize=5, markersize=8)
    ax2.set_xlabel('Sample Size')
    ax2.set_ylabel('Mean Centroid Distance')
    ax2.set_title('Centroid Stability vs Sample Size')
    ax2.set_xscale('log')
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('K-Modes Stability Analysis Across Sample Sizes', fontsize=16)
    plt.tight_layout()
    
    return results_by_size

# Run multi-scale analysis with very small samples for quick testing
print("Running multi-scale stability analysis...")
multi_results = multi_scale_stability_analysis(
    bfrss_raw_df,
    DEMOGRAPHIC_FEATURE_COLUMNS,
    n_clusters=2,
    sample_sizes=[500, 1000, 2000, 5000],  # Small sizes for quick testing
    n_iterations=5
)

In [ ]:
# Define demographic features and health condition targets

# Demographic features from the visualization notebook
DEMOGRAPHIC_FEATURES = [
    'MARITAL',    # Marital status
    'EDUCA',      # Education level
    'RENTHOM1',   # Home ownership
    'VETERAN3',   # Veteran status
    'EMPLOY1',    # Employment status
    'INCOME3',    # Income level
    'SEXVAR',     # Sex/Gender
    '_HISPANC',   # Hispanic ethnicity
    '_IMPRACE',   # Race
]

# Chronic health condition features to predict
# These are common chronic conditions in BRFSS data
CHRONIC_HEALTH_CONDITIONS = [
    'DIABETE4',   # Diabetes
    'CVDINFR4',   # Heart attack
    'CVDCRHD4',   # Coronary heart disease
    'CVDSTRK3',   # Stroke
    'ASTHMA3',    # Asthma
    'CHCSCNC1',   # Skin cancer
    'CHCOCNC1',   # Other cancer
    'CHCKDNY2',   # Kidney disease
    'CHCCOPD3',   # COPD
    'HAVARTH5',   # Arthritis
    'ADDEPEV3',   # Depression
    'DECIDE',     # Cognitive decline
]

# Healthcare access features (for comparison with RQ1)
HEALTHCARE_ACCESS_FEATURES = [
    'HLTHPLN1',   # Health insurance
    'PERSDOC3',   # Personal doctor
    'MEDCOST1',   # Couldn't see doctor due to cost
    'CHECKUP1',   # Time since last checkup
]

print(f"Demographic features ({len(DEMOGRAPHIC_FEATURES)}): {DEMOGRAPHIC_FEATURES}")
print(f"\\nChronic health conditions ({len(CHRONIC_HEALTH_CONDITIONS)}): {CHRONIC_HEALTH_CONDITIONS}")
print(f"\\nHealthcare access features ({len(HEALTHCARE_ACCESS_FEATURES)}): {HEALTHCARE_ACCESS_FEATURES}")

In [ ]:
# Example usage and batch prediction function
def run_multiple_predictions(df, feature_columns, target_columns, 
                            model_params=None, training_params=None):
    """
    Run predictions for multiple target variables using the same feature set.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_columns : list
        List of feature column names
    target_columns : list
        List of target column names to predict
    model_params : dict, optional
        Model parameters
    training_params : dict, optional
        Training parameters
        
    Returns:
    --------
    results : dict
        Dictionary with results for each target variable
    """
    
    all_results = {}
    
    for target_col in target_columns:
        print(f"\\n\\n{'='*80}")
        print(f"RUNNING PREDICTION FOR: {target_col}")
        print(f"{'='*80}")
        
        try:
            result = run_tensorflow_prediction(
                df, feature_columns, target_col, 
                model_params=model_params, 
                training_params=training_params
            )
            
            if result is not None:
                all_results[target_col] = result
                print(f"\\n✓ Successfully completed prediction for {target_col}")
                print(f"  - Accuracy: {result['evaluation']['test_accuracy']:.4f}")
                if 'auc' in result['evaluation']:
                    print(f"  - AUC: {result['evaluation']['auc']:.4f}")
            else:
                print(f"\\n✗ Failed to complete prediction for {target_col}")
                
        except Exception as e:
            print(f"\\n✗ Error predicting {target_col}: {str(e)}")
            continue
    
    # Summary
    print(f"\\n\\n{'='*80}")
    print(f"PREDICTION SUMMARY")
    print(f"{'='*80}")
    print(f"Total targets attempted: {len(target_columns)}")
    print(f"Successful predictions: {len(all_results)}")
    print(f"\\nResults summary:")
    
    for target, result in all_results.items():
        accuracy = result['evaluation']['test_accuracy']
        auc = result['evaluation'].get('auc', 'N/A')
        print(f"  {target:15s} - Accuracy: {accuracy:.4f}, AUC: {auc}")
    
    return all_results

# Quick example of how to use the functions
print("\\n" + "="*60)
print("TENSORFLOW ML PREDICTION SETUP COMPLETE")
print("="*60)
print("\\nAvailable arrays for predictions:")
print(f"\\nDEMOGRAPHIC_FEATURES ({len(DEMOGRAPHIC_FEATURES)} features):")
for i, feature in enumerate(DEMOGRAPHIC_FEATURES):
    print(f"  {i+1:2d}. {feature}")

print(f"\\nCHRONIC_HEALTH_CONDITIONS ({len(CHRONIC_HEALTH_CONDITIONS)} targets):")
for i, condition in enumerate(CHRONIC_HEALTH_CONDITIONS):
    print(f"  {i+1:2d}. {condition}")

print(f"\\nHEALTHCARE_ACCESS_FEATURES ({len(HEALTHCARE_ACCESS_FEATURES)} targets):")
for i, feature in enumerate(HEALTHCARE_ACCESS_FEATURES):
    print(f"  {i+1:2d}. {feature}")

print("\\n" + "="*60)
print("USAGE EXAMPLES:")
print("="*60)
print("""
# Example 1: Single prediction
results = run_tensorflow_prediction(
    bfrss_raw_df, 
    DEMOGRAPHIC_FEATURES, 
    'DIABETE4'  # Predict diabetes
)

# Example 2: Multiple predictions
results = run_multiple_predictions(
    bfrss_raw_df,
    DEMOGRAPHIC_FEATURES,
    ['DIABETE4', 'CVDINFR4', 'ASTHMA3']  # Predict multiple conditions
)

# Example 3: Custom model parameters
custom_model_params = {
    'hidden_layers': [128, 64, 32],
    'dropout_rate': 0.4,
    'learning_rate': 0.0005
}
results = run_tensorflow_prediction(
    bfrss_raw_df, 
    DEMOGRAPHIC_FEATURES, 
    'DIABETE4',
    model_params=custom_model_params
)
""")
print("="*60)

In [ ]:
# Main prediction function that can be called with different feature sets
def run_tensorflow_prediction(df, feature_columns, target_column, 
                             model_params=None, training_params=None,
                             test_size=0.2, random_state=42):
    """
    Complete pipeline for TensorFlow prediction using demographic features.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe (BRFSS data)
    feature_columns : list
        List of feature column names to use for prediction
    target_column : str
        Name of target column to predict
    model_params : dict, optional
        Model parameters (hidden_layers, dropout_rate, learning_rate)
    training_params : dict, optional
        Training parameters (epochs, batch_size, validation_split)
    test_size : float
        Proportion of data to use for testing
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    results : dict
        Dictionary containing model, history, evaluation metrics, and data splits
    """
    
    print(f"\\n{'='*60}")
    print(f"TensorFlow ML Prediction: {target_column}")
    print(f"Features: {feature_columns}")
    print(f"{'='*60}")
    
    # Default parameters
    if model_params is None:
        model_params = {
            'hidden_layers': [64, 32, 16],
            'dropout_rate': 0.3,
            'learning_rate': 0.001
        }
    
    if training_params is None:
        training_params = {
            'epochs': 50,
            'batch_size': 32,
            'validation_split': 0.2,
            'verbose': 1
        }
    
    # Check if target column exists
    if target_column not in df.columns:
        print(f"Error: Target column '{target_column}' not found in dataframe")
        return None
    
    # Check if all feature columns exist
    missing_features = [col for col in feature_columns if col not in df.columns]
    if missing_features:
        print(f"Error: Feature columns not found: {missing_features}")
        return None
    
    # Preprocess data
    print("\\nPreprocessing data...")
    X, y, encoders, scaler = preprocess_data_for_ml(df, feature_columns, target_column)
    
    print(f"Data shape after preprocessing: X={X.shape}, y={y.shape}")
    print(f"Target distribution: {np.bincount(y)}")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    print(f"Train set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    
    # Build model
    print("\\nBuilding neural network...")
    model = build_neural_network(
        input_dim=X_train.shape[1],
        output_dim=1,  # Binary classification
        **model_params
    )
    
    print(f"Model architecture:")
    model.summary()
    
    # Train model
    print("\\nTraining model...")
    history, evaluation = train_and_evaluate_model(
        model, X_train, y_train, X_test, y_test, **training_params
    )
    
    # Print evaluation results
    print_evaluation_report(
        y_test, evaluation['y_pred'], evaluation['y_pred_prob'].flatten(), 
        target_name=target_column
    )
    
    # Create visualizations
    print("\\nGenerating visualizations...")
    
    # Training history
    plot_training_history(history, f"{target_column} - ")
    plt.show()
    
    # Confusion matrix and ROC curve
    plot_confusion_matrix_and_roc(
        y_test, evaluation['y_pred'], evaluation['y_pred_prob'].flatten(),
        f"{target_column} - "
    )
    plt.show()
    
    # Return results
    results = {
        'model': model,
        'history': history,
        'evaluation': evaluation,
        'encoders': encoders,
        'scaler': scaler,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'feature_columns': feature_columns,
        'target_column': target_column
    }
    
    return results

In [ ]:
# Visualization functions for model evaluation
def plot_training_history(history, title_prefix=""):
    """
    Plot training history including loss and accuracy curves.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot training & validation loss
    axes[0].plot(history.history['loss'], label='Training Loss')
    axes[0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0].set_title(f'{title_prefix}Model Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot training & validation accuracy
    axes[1].plot(history.history['accuracy'], label='Training Accuracy')
    axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[1].set_title(f'{title_prefix}Model Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def plot_confusion_matrix_and_roc(y_true, y_pred, y_pred_prob, title_prefix=""):
    """
    Plot confusion matrix and ROC curve for binary classification.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title(f'{title_prefix}Confusion Matrix')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    auc = roc_auc_score(y_true, y_pred_prob)
    
    axes[1].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc:.3f})')
    axes[1].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title(f'{title_prefix}ROC Curve')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def print_evaluation_report(y_true, y_pred, y_pred_prob, target_name="Target"):
    """
    Print comprehensive evaluation report.
    """
    print(f"\\n=== {target_name} Prediction Results ===")
    print(f"Accuracy: {(y_pred == y_true).mean():.4f}")
    
    if len(np.unique(y_true)) == 2:
        auc = roc_auc_score(y_true, y_pred_prob)
        print(f"AUC-ROC: {auc:.4f}")
    
    print(f"\\nClassification Report:")
    print(classification_report(y_true, y_pred))
    
    # Class distribution
    unique, counts = np.unique(y_true, return_counts=True)
    print(f"\\nClass Distribution (Test Set):")
    for class_val, count in zip(unique, counts):
        print(f"  Class {class_val}: {count} ({count/len(y_true)*100:.1f}%)")

In [ ]:
# TensorFlow Neural Network Model Builder
def build_neural_network(input_dim, output_dim=1, hidden_layers=[64, 32, 16], 
                        dropout_rate=0.3, learning_rate=0.001):
    """
    Build a neural network for classification using TensorFlow/Keras.
    
    Parameters:
    -----------
    input_dim : int
        Number of input features
    output_dim : int
        Number of output classes (1 for binary classification)
    hidden_layers : list
        List of hidden layer sizes
    dropout_rate : float
        Dropout rate for regularization
    learning_rate : float
        Learning rate for optimizer
    
    Returns:
    --------
    model : keras.Sequential
        Compiled neural network model
    """
    
    model = keras.Sequential()
    
    # Input layer
    model.add(keras.layers.Dense(hidden_layers[0], activation='relu', 
                                input_shape=(input_dim,)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(dropout_rate))
    
    # Hidden layers
    for units in hidden_layers[1:]:
        model.add(keras.layers.Dense(units, activation='relu'))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dropout(dropout_rate))
    
    # Output layer
    if output_dim == 1:
        # Binary classification
        model.add(keras.layers.Dense(1, activation='sigmoid'))
        loss = 'binary_crossentropy'
        metrics = ['accuracy', keras.metrics.AUC(name='auc')]
    else:
        # Multi-class classification
        model.add(keras.layers.Dense(output_dim, activation='softmax'))
        loss = 'sparse_categorical_crossentropy'
        metrics = ['accuracy']
    
    # Compile model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

# Function to train and evaluate model
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, 
                           epochs=50, batch_size=32, validation_split=0.2,
                           verbose=1):
    """
    Train and evaluate a neural network model.
    
    Returns:
    --------
    history : keras.callbacks.History
        Training history
    evaluation : dict
        Test set evaluation metrics
    """
    
    # Early stopping callback
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    # Reduce learning rate on plateau
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=[early_stopping, reduce_lr],
        verbose=verbose
    )
    
    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)[:2]
    
    # Get predictions
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    evaluation = {
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'y_pred': y_pred,
        'y_pred_prob': y_pred_prob
    }
    
    # Calculate AUC if binary classification
    if len(np.unique(y_test)) == 2:
        evaluation['auc'] = roc_auc_score(y_test, y_pred_prob)
    
    return history, evaluation